In [1]:
import torch

print(torch.cuda.is_available())  # Phải trả về True

True


YOLO V11

In [1]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")  

results = model.train(data="Tu_lam_v2/data.yaml", epochs=250, imgsz=640)

Ultralytics 8.3.107  Python-3.10.0 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=Tu_lam_v2/data.yaml, epochs=250, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes

train: Scanning D:\Downloads\Compressed\Train\Tu_lam_v2\train\labels... 172 images, 3 backgrounds, 0 corrupt:  56%|█████▌    | 172/306 [00:00<00:00, 296.28it/s]


KeyboardInterrupt: 

YOLO V9

In [2]:
from ultralytics import YOLO
import cv2
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# Load YOLO models

emotion_model = YOLO("runs/detect/train15/weights/best.pt")

In [3]:
emotion_mapping = {
    "Angry": "Giận dữ",
    "Contempt": "Khinh thường",
    "Disgust": "Chán ghét",
    "Happy": "Vui vẻ",
    "Neutral": "Bình thường",
    "Sad": "Buồn bã",
    "Surprise": "Ngạc nhiên",
}

# Định nghĩa màu cho từng cảm xúc
emotion_colors = {
    "Angry": (255, 0, 0),      # Đỏ
    "Contempt": (0, 255, 255),  # Cyan
    "Disgust": (128, 0, 128),  # Tím
    "Happy": (0, 255, 0),      # Xanh lá
    "Neutral": (255, 192, 203), # Hồng
    "Sad": (255, 0, 0),        # Đỏ
    "Surprise": (218, 165, 32)   # Vàng
}

In [4]:
font_path = "arial.ttf"
font = ImageFont.truetype(font_path, 20)

In [5]:
# Open webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Chuyển frame sang dạng PIL để vẽ
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_image)

    # Nhận diện cảm xúc
    emotion_results = emotion_model(frame)

    # Xử lý kết quả nhận diện
    for result in emotion_results[0].boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score < 0.5:
            continue
        
        # Lấy tên cảm xúc và màu tương ứng
        english_emotion = emotion_model.names[int(class_id)]
        viet_emotion = emotion_mapping.get(english_emotion, "Không xác định")
        box_color = emotion_colors.get(english_emotion, (128, 128, 128))

        # Vẽ bounding box
        draw.rectangle([x1, y1, x2, y2], outline=box_color, width=3)

        # Hiển thị nhãn tiếng Anh, độ tin cậy và tiếng Việt
        label_text = f"{english_emotion} {score:.1f}"
        viet_text = f"{viet_emotion}"
        
        # Vẽ background và text tiếng Anh
        text_bbox = draw.textbbox((x1, y1 - 25), label_text, font=font)
        draw.rectangle([text_bbox[0]-2, text_bbox[1]-2, text_bbox[2]+2, text_bbox[3]+2], 
                      fill=box_color)
        draw.text((x1, y1 - 25), label_text, font=font, fill=(255, 255, 255))
        
        # Vẽ background và text tiếng Việt phía trên
        viet_bbox = draw.textbbox((x1, y1 - 50), viet_text, font=font)
        draw.rectangle([viet_bbox[0]-2, viet_bbox[1]-2, viet_bbox[2]+2, viet_bbox[3]+2], 
                      fill=box_color)
        draw.text((x1, y1 - 50), viet_text, font=font, fill=(255, 255, 255))

    # Chuyển lại về OpenCV để hiển thị

    
    frame = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    cv2.imshow("Emotion Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 1 Contempt, 1 Neutral, 132.6ms
Speed: 3.3ms preprocess, 132.6ms inference, 3.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Neutral, 1 Sad, 13.6ms
Speed: 3.1ms preprocess, 13.6ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Neutral, 1 Sad, 17.3ms
Speed: 2.7ms preprocess, 17.3ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Neutral, 1 Sad, 18.2ms
Speed: 2.5ms preprocess, 18.2ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Neutral, 1 Sad, 18.4ms
Speed: 3.0ms preprocess, 18.4ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Neutral, 17.8ms
Speed: 2.9ms preprocess, 17.8ms inference, 9.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Neutral, 17.3ms
Speed: 4.7ms preprocess, 17.3ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Neutral, 19.2ms
Speed: 2.9ms preprocess, 19.2ms inferen

In [11]:
!yolo task=detect mode=predict model="runs/detect/train2/weights/best.pt" source="D:\Downloads\Compressed\Train\Data\Data_co_san\20240902_204316(0).jpg"

Ultralytics 8.3.58 🚀 Python-3.12.4 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
YOLO11n summary (fused): 238 layers, 2,583,712 parameters, 0 gradients, 6.3 GFLOPs

image 1/1 D:\Downloads\Compressed\Train\Data\Data_co_san\20240902_204316(0).jpg: 640x480 1 Neutral, 56.0ms
Speed: 4.0ms preprocess, 56.0ms inference, 72.8ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs\detect\predict2
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [3]:
from ultralytics import YOLO
import cv2
import os

# Load mô hình YOLO đã huấn luyện trước
model = YOLO(r"D:\Downloads\Compressed\Train\runs\detect\yolo_v11\weights\last.pt")

# Thư mục chứa ảnh đầu vào
image_folder = "Data/Data_co_san/images"
label_folder = "Data/Data_co_san/labels"

# Đảm bảo thư mục lưu nhãn tồn tại
os.makedirs(label_folder, exist_ok=True)

# Duyệt qua các ảnh trong thư mục
for image_name in os.listdir(image_folder):
    if image_name.endswith((".jpg", ".png", ".jpeg")):
        image_path = os.path.join(image_folder, image_name)
        results = model(image_path)  # Dự đoán ảnh

        # Tạo file nhãn cho ảnh
        label_path = os.path.join(label_folder, image_name.replace(".jpg", ".txt").replace(".png", ".txt"))
        
        with open(label_path, "w") as f:
            for result in results:
                for box in result.boxes:
                    x_center, y_center, width, height = box.xywhn[0]  # Lấy tọa độ chuẩn hóa
                    class_id = int(box.cls[0])  # ID của class
                    f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
        
        print(f"Labeled: {image_name}")

for result in results:
    im = result.plot()  # Vẽ bounding box lên ảnh
    cv2.imshow("YOLO Labels", im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



image 1/1 d:\Downloads\Compressed\Train\Data\Data_co_san\images\1000001648.png: 640x480 1 Neutral, 1 Surprise, 72.0ms
Speed: 6.0ms preprocess, 72.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 480)
Labeled: 1000001648.png

image 1/1 d:\Downloads\Compressed\Train\Data\Data_co_san\images\116335387_578578579491829_1958339432913620680_n.png: 640x352 1 Surprise, 75.0ms
Speed: 3.0ms preprocess, 75.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 352)
Labeled: 116335387_578578579491829_1958339432913620680_n.png

image 1/1 d:\Downloads\Compressed\Train\Data\Data_co_san\images\116443743_578578439491843_1979972365347154801_n.png: 448x640 2 Sads, 77.0ms
Speed: 4.0ms preprocess, 77.0ms inference, 3.0ms postprocess per image at shape (1, 3, 448, 640)
Labeled: 116443743_578578439491843_1979972365347154801_n.png

image 1/1 d:\Downloads\Compressed\Train\Data\Data_co_san\images\116604893_578578209491866_8728610359751953131_n.png: 640x640 1 Neutral, 63.0ms
Speed: 7.0ms p